#Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Loading Pre-Trained BERT

In [ ]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 2.9MB/s 


# Load Transformers

In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 14.1MB/s 
     |████████████████████████████████| 890kB 18.3MB/s 
     |████████████████████████████████| 3.0MB 25.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=57ef66a21eab5e4b7a9cddfc1fa9cf952d61aa2a534db03e56c4a3975823529e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


#Load the utilities

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from sklearn.model_selection import train_test_split
import re
import numpy as np
import io
import time

In [ ]:
path_to_file = "drive/My Drive/english_telugu_data.txt"

pre-process

In [ ]:
def preprocess_sentence(w):
  w = w.lower()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = '[CLS] ' + w + ' [SEP]'
  return w

loading the dataset

In [ ]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  word_pairs = [[preprocess_sentence(w) for w in l.split('++++$++++')]  for l in lines[:num_examples]]
  return zip(*word_pairs)

In [ ]:
en, te = create_dataset(path_to_file, num_examples=10000)

In [ ]:
en[0],te[0]

('[CLS] his legs are long .  [SEP]',
 '[CLS] అతని కాళ్ళు పొడవుగా ఉన్నాయి .  [SEP]')

#Loading the BERT Tokenizer

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 995526/995526 [00:00<00:00, 12524825.00B/s]


In [ ]:
from transformers import DistilBertTokenizer
tokenizer2 = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [ ]:
target_tok=[]
input_tok=[]
input_tensor=[]
target_tensor=[]
for i in range(len(en)):
  tokenized_text1 = tokenizer.tokenize(en[i])
  tokenized_text2 = tokenizer.tokenize(te[i])
  input_tok.append(tokenized_text1)
  target_tok.append(tokenized_text2)

In [ ]:
for j in range(len(en)):
  indexed_tokens_eng = tokenizer.convert_tokens_to_ids(input_tok[j])
  indexed_tokens_tel = tokenizer.convert_tokens_to_ids(target_tok[j])
  input_tensor.append(indexed_tokens_eng)
  target_tensor.append(indexed_tokens_tel)

In [ ]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [ ]:
print(max_length(input_tensor))
print(max_length(target_tensor))

35
67


In [ ]:
max_length_inp=max_length(input_tensor)
max_length_targ=max_length(target_tensor)

In [ ]:
input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,
                                                         padding='post')
target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,
                                                         padding='post')

splitting the data for training and validation

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [ ]:
def convert(tensor):
  for t in tensor:
    if t!=0:
      print ("%s ----> %s" % (t, tokenizer.convert_tokens_to_ids([t])))

In [ ]:
print ("Input Language; index to word mapping")
convert(input_tok[0])
print ()
print ("Target Language; index to word mapping")
convert(target_tok[0])

Input Language; index to word mapping
[CLS] ----> [101]
his ----> [10226]
legs ----> [51863]
are ----> [10301]
long ----> [11695]
. ----> [119]
[SEP] ----> [102]

Target Language; index to word mapping
[CLS] ----> [101]
అతని ----> [62522]
క ----> [1201]
##ా ----> [15735]
##ళ్ళు ----> [92090]
ప ----> [1220]
##ొ ----> [111340]
##డ ----> [36008]
##వు ----> [27850]
##గా ----> [12540]
ఉన్నాయి ----> [11251]
. ----> [119]
[SEP] ----> [102]


In [ ]:
v=np.unique(input_tensor)
print(v[-1])
w=np.unique(target_tensor)
print(w[-1])

110790
111341


In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 32
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 64
units = 256
vocab_inp_size = v[-1]+1
vocab_tar_size = w[-1]+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 35]), TensorShape([32, 67]))

Encoder

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (32, 35, 256)
Encoder Hidden state shape: (batch size, units) (32, 256)


Attention method

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (32, 256)
Attention weights shape: (batch_size, sequence_length, 1) (32, 35, 1)


Decoder

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, st, att = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))
print ('Decoder output shape: {}'.format(st.shape))
print ('Decoder output shape: {}'.format(att.shape))

Decoder output shape: (batch_size, vocab size) (32, 111342)
Decoder output shape: (32, 256)
Decoder output shape: (32, 35, 1)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

Training Function

In [ ]:
#@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims(tokenizer.convert_tokens_to_ids(['[CLS]']) * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.3983
Epoch 1 Batch 100 Loss 1.3814
Epoch 1 Batch 200 Loss 1.4441
Epoch 1 Loss 1.4792
Time taken for 1 epoch 234.63261675834656 sec

Epoch 2 Batch 0 Loss 1.2974
Epoch 2 Batch 100 Loss 1.2634
Epoch 2 Batch 200 Loss 1.3143
Epoch 2 Loss 1.3523
Time taken for 1 epoch 232.5425319671631 sec

Epoch 3 Batch 0 Loss 1.4748
Epoch 3 Batch 100 Loss 1.2264
Epoch 3 Batch 200 Loss 1.2505
Epoch 3 Loss 1.1970
Time taken for 1 epoch 230.7812294960022 sec

Epoch 4 Batch 0 Loss 1.0561
Epoch 4 Batch 100 Loss 1.1066
Epoch 4 Batch 200 Loss 0.9813
Epoch 4 Loss 1.0596
Time taken for 1 epoch 230.39495611190796 sec

Epoch 5 Batch 0 Loss 1.1285
Epoch 5 Batch 100 Loss 1.0407
Epoch 5 Batch 200 Loss 1.0009
Epoch 5 Loss 0.9901
Time taken for 1 epoch 230.4363534450531 sec

Epoch 6 Batch 0 Loss 1.0189
Epoch 6 Batch 100 Loss 0.8861
Epoch 6 Batch 200 Loss 0.9767
Epoch 6 Loss 0.9454
Time taken for 1 epoch 234.4301781654358 sec

Epoch 7 Batch 0 Loss 0.9016
Epoch 7 Batch 100 Loss 0.9356
Epoch 7 Batch 20

Evaluation and Translate

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  input_tokens=tokenizer.tokenize(sentence)
  inputs=tokenizer.convert_tokens_to_ids(input_tokens)

  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''
  result2=[]

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims(tokenizer.convert_tokens_to_ids(['[CLS]']), 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()
    result2.append(predicted_id)

    result += str(tokenizer.convert_ids_to_tokens([predicted_id])) + ' '

    if tokenizer.convert_ids_to_tokens([predicted_id]) == tokenizer.convert_ids_to_tokens([102]):
      return result, sentence, attention_plot, result2

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot, result2

In [ ]:
def translate(sentence):
  result, sentence, attention_plot, result2 = evaluate(sentence)
  predicted_text=tokenizer2.decode(result2)
  predicted_text=predicted_text.replace(" [SEP]","")
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(predicted_text))

In [ ]:
translate(u'She knows nothing about your family.')

Input: [CLS] she knows nothing about your family .  [SEP]
Predicted translation: మీ కుటుంబం గురించి ఆమెకు ఎప్పుడూ నాకు తెలియదు.


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
reference = [[ "మీ", "కుటుంబం" ,"గురించి" ,"ఆమెకు","ఏమీ" ,"తెలియదు"]]
test = ["మీ" ,"కుటుంబం" ,"గురించి", "ఆమెకు", "ఎప్పుడూ" ,"తెలియదు"]
score = sentence_bleu(reference,test)
print(score)

0.537284965911771
